# Buffer Pool Assignment

In this class we assume that the primary storage location of the database is on disk.

The goal of this programming project is to implement a buffer pool in your storage manager. 
The buffer pool is responsible for moving physical pages back and forth from main memory to disk. 

You are given the following components:
- Disk Manager: It is responsible to create, read, write and delete pages on disk
- Buffer Pool Manager: It is responsible for fetching database pages from the DiskManager and storing them in memory. 
- Replacer: It keeps track of when Page are accessed so that it can decide which one to evict when it must free a frame to make room for copying a new physical page from disk.
- Page: It is the representation of in-memory pages

You will need to implement/extend the following two components in your storage manager:

- Least recently used (LRU) Replacement Policy: it discards the least recently used page first. This algorithm requires keeping track of what was used when. 
- Buffer Pool Manager


In [3]:
class Page:
    """
    A class to represent a in-memory page.
    A page object maintains a counter for the number of threads that have pinned that page, and keeps track of 
    whether it is dirty or not. 

    Attributes
    ----------
    page_id : int
        the page object's identifier
    pin_count : int
        counter of how many threads have pin this page
    dirty : bool
        tracks if a page has been modified by a thread

    Methods
    -------
    incrementPinCount():
        increments the page's pin counter.
    decrementPinCount():
        decrements the page's pin counter.
    isDirty():
        returns if a page has been modified.
    getPinCount():
        return the page's pin counter.
    """
    def __init__(self, id):
        self.page_id = id
        self.pin_count = 0
        self.dirty = False

    def incrementPinCount(self):
        self.pin_count += 1

    def decrementPinCount(self):
        self.pin_count -= 1

    def isDirty(self):
        return self.dirty

    def getPinCount(self):
        return self.pin_count

In [4]:
class DiskManager:
    """
    A class to represent the disk manager
    The Disk manager is responsible to read and write data to disk

    Attributes
    ----------
    pages : array
        list of page_id read from disk
    invalid : array
        pages that have been deleted from disk
        
    Methods
    -------
    readPage(page_id):
        reads the page from disk with id = page_id.
    writePage(page):
        writes a page to disk.
    deletePage():
        deletes a page in disk and set the page_id as invalid.
    createPage():
        creates a page in disk.
    """
        
    def __init__(self):
        self.pages = []
        self.invalid = []
        
    def readPage(self, page_id):
        """Reads the page from disk with id = page_id..

        Parameters
        ----------
        page_id : int
            The identifier of the page that is being requested

        Raises
        ------
        Error
           If the page_id is invalid.
        """
        print('read a page from disk', page_id)
        if page_id not in self.invalid:
            if page_id not in self.pages:
                self.createPage(page_id)
            self.pages.append(page_id)
            return Page(page_id)
        else:
            print('Page has been deleted from disk')
            return None
    
    def createPage(self, page_id):
        print('create new page in disk', page_id)
        pass
    
    def writePage(self, page):
        print('write a page to disk', page.page_id)
        pass
    
    def deletePage(self, page):
        print('delete page from disk', page.page_id)
        page_id = page.page_id
        self.pages.remove(page_id)
        self.invalid.append(page_id)
        
    

In [5]:
class Replacer:
    """
    A class to represent the page replacer
    The Replacer keeps track of when Page objects are accessed so that it can decide which one to 
    evict when it must free a frame to make room for copying a new physical page from disk.


    Methods
    -------
    victim():
        return which frame should be evicted from the BufferPool. 
    pin(page_id):
        pin a page in the BufferPool
    unpin():
        unpin a page in the buffer pool
    replacerSize():
        returns the number of frames that are currently in the Replacer.

    """
    def __init__(self):
        pass

    def victim(self):
        pass

    def pin(self, page_id):
        pass
    
    def unpin(self, page_id):
        pass

    def replacerSize(self):
        pass

In [6]:
class LRUReplacer(Replacer):
    """
    A subclass of Replacer that implement a specific replacement strategy.
    The LRU replacer discards the least recently used page first. 
    This algorithm requires keeping track of what page objects was used when, so that it can decide 
    which one to evict when it must free a frame to make room for copying a new physical page from disk.

    Attributes
    ----------
    free_frames : array
        frames that can be evited, if needed. When initialized, there is no free frame in it. Only unpinned 
        frames can be added to the free_frames list.
   
    Methods
    -------
    victim():
        identifies the frame from the free_frames list that was accessed the least recently. if there is such a frame, then store its 
        contents in the output parameter and return true. if there is no frame to be evicted (free_frames list is
        empty, then return False.
    pin(page_id):
        when a page is pinned, its corresponding frame in the Buffer Pool cannot be evicted until its pin counter 
        is 0 again. This funcion removes the frame containing the pinned page from the free_frames list in the Replacer
    unpin():
        when the pin_count of a page becomes 0, the frame can be unpined. This method should add the frame 
        containing the unpinned page into the Replacer free_frames list.
    getFreeFrames():
        return free_frames list
    """
    
    def __init__(self):
        super().__init__()
        self.free_frames = []
        
    def getFreeFrames(self):
        return self.free_frames
    
    def pin(self, page_id):
        print('page to pin', page_id)
        ##ADD YOUR CODE HERE
        if page_id in self.getFreeFrames():      #make sure page exists first 
            self.getFreeFrames().remove(page_id) #removes page from Free Frames list 
            return True 
        else:
            return False

    def unpin(self, page_id):
        print('page to unpin', page_id)
        ##ADD YOUR CODE HERE
        self.getFreeFrames().append(page_id) #insert page into Free Frames list 
        return False
    
    ## delete from memory and flush to disk
    def victim(self):
        print('try to evict page')
        ##ADD YOUR CODE HERE
        if not self.getFreeFrames():
            return False 
        else:
            victim = self.getFreeFrames()[-1]   #find LRU frame from Free Frames list (grab the last frame in the list)
            self.getFreeFrames().remove(victim) #evict frame from Free Frames list
            return victim #return LRU frame (it will be flushed to disk in the BufferPoolManager class since there is no reference to the DiskManager in the LRU Replacer)
        
    def replacerSize(self):
        return len(self.free_list)

In [17]:
class BufferPoolManager:
    """
    A class to represent the BufferPoolManager
    The BufferPoolManager is responsible for fetching database pages from the DiskManager and 
    storing them in memory. 
    
    The BufferPoolManager must write the contents of a dirty Page back to disk before that object can be reused. It 
    writes dirty pages out to disk when it is either explicitly instructed to do so or when it needs to evict a page to make space for a new page.
    
    The BufferPoolManager is not allowed to free a Page that is pinned. 
    
    This BufferPoolManager implementation will use the LRUReplacer class. 
    

    Attributes
    ----------
    buffer_pool : array
        list with page objects
    page_table: array
        list of page_ids current in the buffer pool 
    buffer_total_no_of_frames: int
        total of frames that the buffer pool can hold
    disk_manager: DiskManager
        diskManager object
    replacer: LRUReplacer
        replacer object
   
    Methods
    -------
    getBufferPoll():
        returns all page objects in the buffer pool
    getPageTable():
        return the index of all pages objects currently in the buffer pool
    getDiskManager():
        return disk manager object
    getReplacer():
        return replace object
    fetchPage(page_id):
        returns a page stored in memory. If the page is not in memory, then the page must be read from the disk.
    newPage(page_id):
        reads a page from the disk using the disk manager
    deletePage(page_id):
        deletes a page from the buffer pool and from the disk
    unpinPage(page_id, is_dirty):
        uping a page so the replacer knows it is a free frame that can be evited if needed.
    flushPage(page_id):
        flushs a page with id = page_id from the buffer pool to disk regardless of its pin status.
    flushAllPages():
        flush all pages from the buffer pool to disk regardless of its pin status.
    
    """
    
    def __init__(self, no_of_frames):
        self.buffer_pool = []
        self.page_table = []
        self.disk_manager = DiskManager()
        self.replacer = LRUReplacer()
        self.buffer_total_no_of_frames = no_of_frames
        
    def getBufferPool(self):
        return self.buffer_pool 
    
    def getPageTable(self):
        return self.page_table
    
    def getReplacer(self):
        return self.replacer
    
    def getDiskManager(self):
        return self.disk_manager
    
    def getNumFrames(self):
        return self.buffer_total_no_of_frames
    
    def fetchPage(self, page_id):
        # fetch page from the buffer pool, if not in memory than get from disk
        #  1. search the page table for the requested page_id.
        #  1.1   if page exists in the bufferpool, imcrements its pin counter, pin it and return it immediately.
        #  1.2   if page is not in the bufferpool, fetch the page from disk using the disk manager and pin it. If the bufferpool is full, find a replacement page using the replacer.  If no pages can be evited (i.e., all pages are pinned), print a error message  
        #  2. if you need to evict a page, check if it is dirty. If so, write it back to the disk.
        #  3. delete the evicted page from the page table and insert the page you fecthed from disk.
        #  4. update the page metadata, and then return a pointer to it.
        print('fetch page from the bufferpool', page_id)
        ##ADD YOUR CODE HERE
        if page_id in self.getPageTable(): #1 and 1.1
            index = self.getPageTable().index(page_id) #get index of page in buffer pool from page table 
            page = self.getBufferPool()[index] #get page from buffer pool 
            page.incrementPinCount() #increment pin count of page 
            self.getReplacer().pin(page_id) #pin page, update free frames list 
            return page #return the page 
        else: #1.2
            page = self.getDiskManager().readPage(page_id) #get page from disk using disk manager
            if page is not None:
                page.incrementPinCount() #increase pin counter
                self.getReplacer().pin(page_id) #pin it, update free frames 
                if len(self.getBufferPool()) == self.getNumFrames(): #if buffer pool is FULL 
                    victim = self.getReplacer().victim()#find replacement using replacer
                    if victim is False:
                        print('error: no pages can be evicted from free frames')
                    else:
                        index = self.getPageTable().index(victim) #get index of victim from page table
                        if self.getBufferPool()[index].isDirty(): #2. check if evicted page is dirty
                            self.flushPage(victim) #write dirty page back to disk    
                        self.getPageTable().remove(victim) #3. remove victim from page table
                        victim_page = self.getBufferPool()[index] #page to be removed from pool
                        self.getBufferPool().remove(victim_page) #3. remove victim from buffer pool
                        self.getBufferPool().append(page) #there is room, insert into buffer pool
                        self.getPageTable().append(page_id) #there is room, insert into page table
                        return page
                else:
                    self.getBufferPool().append(page) #there is room, insert into buffer pool
                    self.getPageTable().append(page_id) #there is room, insert into page table
                    return page 
   
    def newPage(self, page_id):
        print('page not in bufferpool, read from disk', page_id)
        #fetch page from the disk manager and put in the buffer pool. 
        #checks if page_id is valid in disk, if valid increment its pin count, pin it and return the page. Otherwise return a error message and page = None.
        ##ADD YOUR CODE HERE
        page = self.getDiskManager().readPage(page_id)
        if page is None:
            print('error: page = None')
            return None
        else:
            page.incrementPinCount() #increment pin count 
            self.getBufferPool().append(page) #add page to buffer pool
            self.getPageTable().append(page_id) #add page to page table 
            self.getReplacer().pin(page_id) #remove from free frame list 
            return page 
    
    def deletePage(self, page_id):
        print('delete page from bufferpool', page_id)
        #only delete the pages that are not pinned. if pinned print a error message and return False
        ##ADD YOUR CODE HERE
        index = self.getPageTable().index(page_id)
        page = self.getBufferPool()[index]
        if page.getPinCount() == 0: #make sure page is not pinned
            self.getBufferPool().remove(page) #remove from buffer pool 
            self.getPageTable().remove(page_id) #remove from page table 
            self.getDiskManager().deletePage(page) #delete from disk 
            return True
        else:
            print('page is pinned, cannot be deleted')
            return False
   
    def unpinPage(self, page_id, is_dirty):
        print('unpin page in bufferpool', page_id)
        #decrements the page pin counter and update page "dirtyness" with the value of is_dirty parameter".
        #if pin_count == 0 then unpin the page.
        ##ADD YOUR CODE HERE
        index = self.getPageTable().index(page_id) #get index of page in buffer pool from page table
        page = self.getBufferPool()[index] #get page from buffer pool
        page.decrementPinCount() #decrement pin counter 
        page.dirty = is_dirty #update page "dirtyness"
        if page.getPinCount() == 0: 
            self.getReplacer().unpin(page_id) #unpin page in replacer, update free frames
            return True
        else:
            return False 
        
    def flushPage(self, page_id):
        print('flush page', page_id)
        #write page to disk and update its metadata
        ##ADD YOUR CODE HERE
        index = self.getPageTable().index(page_id)
        page = self.getBufferPool()[index]
        self.getDiskManager().writePage(page)
        page.is_dirty = False #page has been written to disk so it is no longer "dirty", i.e the buffer and disk page have same contents

    def flushAllPages(self):
        print('flush all pages')
        #write all pages to disk and update metadata
        ##ADD YOUR CODE HERE
        pages = self.getBufferPool()
        for page in pages:
            self.getDiskManager().writePage(page)
            page.is_dirty = False #page has been written to disk so it is no longer "dirty", i.e the buffer and disk page have same contents

In [ ]:
bpm = BufferPoolManager(16) #1. create a buffer pool that can hold 16 frames 

for i in range(0,16): #2. add 0..15 to the buffer pool
    bpm.newPage(i)

print('page table: ', bpm.getPageTable()) #3. print the buffer pool page table 

print('free frames: ', bpm.getReplacer().getFreeFrames()) #4. print replacer free frames list (this should be empty)

page = bpm.fetchPage(14) #5. fetch patch 14

print('page id:', page.page_id) #5. print page id 

print('pin counter: ', page.pin_count) #6. print pin count

page = bpm.fetchPage(16) #7. fetch page 16, this DOES NOT WORK because the buffer pool is FULL and the free frames list is EMPTY, so there is no room in the buffer pool and no pages to evict to make room for 16

print('page table: ', bpm.getPageTable()) #8. print the buffer pools page table

bpm.unpinPage(14, False) #9. unpin page 14, page 14 not dirty

page = bpm.fetchPage(16) #10. try again to fetch page 16, this still does not work for the same reason as the first time, even though we unpinned 14, it was pinned twice so it is not in the free frames list

print('free frames: ', bpm.getReplacer().getFreeFrames()) #11. print the replacer free frame list 

bpm.unpinPage(14, False) #12. unpin page 14 again, page 14 not dirty

print('free frames: ', bpm.getReplacer().getFreeFrames()) #13. print the replacer free frame list 

page = bpm.fetchPage(16) #14. try again to fetch page 16, this WORKS because 14 was in the free frames list, so even though the buffer was full it was removed from the buffer because it was not pinned, making room for 16 to be pulled from disk

print('page table: ', bpm.getPageTable()) #15. print buffer pool page table

bpm.unpinPage(9, True) #16. unpin page 9 and 12, they are both dirty
bpm.unpinPage(12, True)

page = bpm.fetchPage(14) #17. fetch page 14 from buffer pool and print page id. Page 12 was the LRU page, it was dirty so it was flushed to disk before being removed from buffer pool

print('page id: ', page.page_id)

bpm.deletePage(5) #18. delete page 5, this does not work because page 5 is pinned

bpm.unpinPage(5, False) #19. unpin page 5, page 5 not dirty

bpm.deletePage(5) #20. try to delete page 5 again, this works because page 5's pin count was 0

print('page table: ', bpm.getPageTable()) #21. print page table 

bpm.fetchPage(5) #22. fetch page 5

# Testing

1. Create a BufferPool that can hold 16 frames

2. Add page 0..15 to the BufferPool

3. Print the Buffer's Pool PageTable

4. Print the replacer free frames list

5. Fetch page #14 from your bufferPool and prints the page_id

6. Print page #14 pin counter

7. Fetch page #16 from your bufferPool and prints the page_id. Did it work? Why?

8. Print the Buffer's Pool PageTable

9. Unpin page #14. Page #14 is not dirty

10. Try again to fetch page #16. Did it work? Why?

11. Print the replacer free frame

12. Unpin page #14 again. Page 14 is not dirty

13. Print the replacer free frame

14. Try again to fetch page #16.Did it work? Why?

15. Print the Buffer's Pool PageTable

16. Unpin page #9 and #12. They are both dirty

17. Fetch page #14 from your bufferPool and prints the page_id. What happened with the page that was replaced to make room for page #14?

18. Delete page #5. Did it work? Why?

19. Unpin page #5. The page is not dirty

20. Try to delete page #5 again. Did it work?

21. Print page table

22. Fetch page #5